<a href="https://colab.research.google.com/github/joycenerd/yolov5-svhn-detection/blob/main/inference.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Step 0: Use GPU

In [ ]:
# Make sure you use the Colab GPU to run the testing phase

import torch
torch.cuda.is_available()
!nvidia-smi

Tue Nov 16 12:07:52 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 495.44       Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla K80           Off  | 00000000:00:04.0 Off |                    0 |
| N/A   35C    P8    28W / 149W |      3MiB / 11441MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

# Step 1: Git clone your project

In [ ]:
!git clone https://github.com/joycenerd/yolov5-svhn-detection.git

Cloning into 'yolov5-svhn-detection'...
remote: Enumerating objects: 106, done.
remote: Counting objects: 100% (106/106), done.
remote: Compressing objects: 100% (83/83), done.
remote: Total 106 (delta 24), reused 101 (delta 22), pack-reused 0
Receiving objects: 100% (106/106), 177.05 KiB | 2.53 MiB/s, done.
Resolving deltas: 100% (24/24), done.


# Step 2: Install your requirements

In [ ]:
%cd yolov5-svhn-detection/yolov5/
!pip install -r requirements.txt
!pip install googledrivedownloader

/content/yolov5-svhn-detection/yolov5
     |████████████████████████████████| 596 kB 5.3 MB/s 
  Attempting uninstall: PyYAML
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13


In [ ]:
import argparse
import os
import sys
from pathlib import Path

import cv2
import torch
import torch.backends.cudnn as cudnn

ROOT = os.getcwd()  # YOLOv5 root directory
print(ROOT)
if str(ROOT) not in sys.path:
    sys.path.append(str(ROOT))  # add ROOT to PATH
ROOT = Path(os.path.relpath(ROOT, Path.cwd()))  # relative

from models.common import DetectMultiBackend
from utils.datasets import IMG_FORMATS, VID_FORMATS, LoadImages, LoadStreams
from utils.general import (LOGGER, check_file, check_img_size, check_imshow, check_requirements, colorstr,
                           increment_path, non_max_suppression, print_args, scale_coords, strip_optimizer, xyxy2xywh)
from utils.plots import Annotator, colors, save_one_box
from utils.torch_utils import select_device, time_sync

import time
from tqdm import tqdm
from google_drive_downloader import GoogleDriveDownloader as gdd

/content/yolov5-svhn-detection/yolov5


# Step 3: Load model


In [ ]:
weights=['/content/ep54_mAP495.pt']
device=select_device('0')
model=DetectMultiBackend(weights,device=device,dnn=False)
stride, names, pt, jit, onnx = model.stride, model.names, model.pt, model.jit, model.onnx
imgsz = check_img_size(imgsz=640, s=stride)  # check image size

YOLOv5 🚀 0aad3e6 torch 1.10.0+cu111 CUDA:0 (Tesla K80, 11441MiB)

Fusing layers... 
Model Summary: 213 layers, 7037095 parameters, 0 gradients, 15.9 GFLOPs


# Step 4: Wget testing data

In [ ]:
# 1. Download the testing data

gdd.download_file_from_google_drive(file_id='1Fm-avdeNgzhPxhvia0iw9yZzcoOggy7I',
                                    dest_path='./test.zip',
                                    unzip=True)
# Dataloader
source='test'
dataset = LoadImages(source, img_size=imgsz, stride=stride, auto=pt and not jit)
bs = 1  # batch_size
print(f"Test data size: {len(dataset)}")

Unzipping...Done.
Test data size: 13068


# Step 5: Run benchmark

In [ ]:
# Test your inference time
TEST_IMAGE_NUMBER = 100 # This number is fixed.

dt, seen = [0.0, 0.0, 0.0], 0
conf_thres=0.25
iou_thres=0.45
classes=None
agnostic_nms=False
max_det=1000
half=False
cnt=0

start_time=time.time()
for path, im, im0s, vid_cap, s in dataset:
    # t1 = time_sync()
    im = torch.from_numpy(im).to(device)
    im = im.half() if half else im.float()  # uint8 to fp16/32
    im /= 255  # 0 - 255 to 0.0 - 1.0
    if len(im.shape) == 3:
        im = im[None]  # expand for batch dim
    # t2 = time_sync()
    # dt[0] += t2 - t1

    # Inference
    # visualize = increment_path(save_dir / Path(path).stem, mkdir=True) if visualize else False
    pred = model(im, augment=False, visualize=False)
    # t3 = time_sync()
    # dt[1] += t3 - t2

    # NMS
    pred = non_max_suppression(pred, conf_thres, iou_thres, classes, agnostic_nms, max_det=max_det)
    # dt[2] += time_sync() - t3
    cnt+=1
    if cnt==TEST_IMAGE_NUMBER:
      break
    
end_time  = time.time()
print("\nInference time per image: ", (end_time - start_time) / TEST_IMAGE_NUMBER)


Inference time per image:  0.02215820550918579
